## Class Imbalance
### Objectives
1. Why class imbalances are a problem
1. Naive oversampling/undersampling
1. Using the “class_weight” parameter (review)
1. Advanced methods (SMOTE and ADASYN)

A classification task with a significant difference in proportionality between classes is said to suffer from a Class Imbalance. 

This is an extremely common problem in real-world settings, and can add significant complexity to the problem. 

<img src="http://harishsivasubramanian.com/wp-content/uploads/2016/09/class.png" width=300>

### Solution 1 - Cost-Sensitive Learning

By default, classifiers in sklearn have a `class_weight` parameter. This is typically set to `None`, which results in a uniform class weight distribution.  

Can also be set to ”balanced”, which computes weights using the following code:  
`n_samples / (n_classes * np.bincount(y))`  

Weights can also be set manually and passed in as a dictionary, where the weight value determines how much the model should be penalized for getting each class wrong. For example: 

`class_weight = {0: 0.00001, 1: 0.99999}`


### Solution 2: Naive Over/Undersampling

Naive Oversampling/Undersampling are also common ways of dealing with class imbalances. These techniques are not mutually exclusive, and can both be used together. 

<img src="https://i.stack.imgur.com/FEOjd.jpg" width=350>

### Advanced Methods: SMOTE & ADASYN

A more advanced approach to dealing with class imbalances in Synthetic Data Generation.  The two most popular algorithms for this are SMOTE and ADASYN. 

These methods start by computing the K-nearest neighbors for in the minority class for each point in the minority class. They then draw a straight line between them, and pick random points on those lines. 

ADASYN is an improvement on the SMOTE algorithm, because it adds a bit of noise to the points, so they are not linearly correlated with the two real minority samples at each end of the line. 

<img src="https://miro.medium.com/max/1400/1*6UFpLFl59O9e3e38ffTXJQ.png" width=400>

## Implementation & Comparison

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
from sklearn.model_selection import train_test_split

In [3]:
bankdata = pd.read_csv('https://raw.githubusercontent.com/matbesancon/BankNotes/master/data_banknote_authentication.txt', header=None)
bankdata.head()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [4]:
# our data doesn't have header, so we will manually add that on 
headers = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"]
bankdata.columns = headers

bankdata.head()

,Variance,Skewness,Curtosis,Entropy,Class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [5]:
bankdata.Class.value_counts()

0    762
1    610
Name: Class, dtype: int64

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
accuracy_score(bankdata.Class.values, np.zeros(bankdata.Class.shape))

0.5553935860058309

In [8]:
762/(762+610)

0.5553935860058309

In [9]:
from sklearn.metrics import f1_score
f1_score(bankdata.Class.values, np.zeros(bankdata.Class.shape))

0.0

__Train-Test-Split__

In [15]:
X = bankdata.drop('Class', axis=1)  
y = bankdata['Class'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1029, stratify=y)

In [16]:
y_train.value_counts()

0    609
1    488
Name: Class, dtype: int64

In [17]:
609/488

1.2479508196721312

In [18]:
y_test.value_counts()

0    153
1    122
Name: Class, dtype: int64

In [19]:
153/122

1.2540983606557377

__Scaling Data__

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

__Class Imbalance__

In [1]:
!pip install imbalanced-learn

     |████████████████████████████████| 174kB 3.3MB/s eta 0:00:01


In [22]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

In [25]:
# How should we implement
ros = RandomOverSampler(random_state=0)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

In [26]:
y_train.value_counts()

0    609
1    488
Name: Class, dtype: int64

In [27]:
y_train_ros.value_counts()

1    609
0    609
Name: Class, dtype: int64

In [28]:
# How should we implement
rus = RandomUnderSampler(random_state=0)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [30]:
y_train_rus.value_counts()

1    488
0    488
Name: Class, dtype: int64

In [32]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.9672727272727273

In [33]:
lr.fit(X_train_ros, y_train_ros)
lr.score(X_test, y_test)

0.9672727272727273

In [34]:
lr.fit(X_train_rus, y_train_rus)
lr.score(X_test, y_test)

0.9709090909090909

In [35]:
# How should we implement
smote = SMOTE(random_state=0)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

adasyn = ADASYN(random_state=0)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)

In [36]:
y_train.value_counts()

0    609
1    488
Name: Class, dtype: int64

In [37]:
y_train_smote.value_counts()

1    609
0    609
Name: Class, dtype: int64

In [38]:
y_train_adasyn.value_counts()

1    609
0    609
Name: Class, dtype: int64

In [39]:
lr.fit(X_train_smote, y_train_smote)
lr.score(X_test, y_test)

0.9672727272727273

In [40]:
lr.fit(X_train_adasyn, y_train_adasyn)
lr.score(X_test, y_test)

0.9672727272727273

In [41]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.9672727272727273

In [42]:
f1_score(y_test, lr.predict(X_test))

0.9638554216867471